In [1]:
import pandas as pd
from duckdb import query
import pandas as pd
import numpy as np
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
pd.set_option('mode.chained_assignment', None)

In [2]:
word = pd.read_pickle('../../pickles/word.pkl')
pars = pd.read_pickle('../../pickles/pars.pkl')
lexn = pd.read_pickle('../../pickles/lexn.pkl')
pdgm = pd.read_pickle('../../pickles/pdgm.pkl')
book = pd.read_pickle('../../pickles/book.pkl')
chap = pd.read_pickle('../../pickles/chap.pkl')
vers = pd.read_pickle('../../pickles/vers.pkl')
gnt = pd.read_pickle('../../pickles/gnt.pkl')

In [6]:
df=word[word.word_id.between('19010101','19010499')]
df.head(10)[['word_greek','word_english','word_pars_id','word_lexn_id']]

,word_greek,word_english,word_pars_id,word_lexn_id
115591,Πολυμερῶς,In many portions,ADV,4806
115592,καὶ,and,CONJ,0001
115593,πολυτρόπως,in many ways,ADV,4809
115594,πάλαι,long ago,ADV,1417
115595,ὁ,<the>,ART-NMS,0000
115596,θεὸς,God,NOUN-NMS,0013
115597,λαλήσας,having spoken,V-AAP-NMS,0056
115598,τοῖς,to the,ART-DMP,0000
115599,πατράσιν,fathers,NOUN-DMP,0044
115600,ἐν,in,PREP,0005


### functions

In [31]:
def back_to_back_words(w1, w2):
    x = False
    verses = []
    for i,id in enumerate(word.word_lexn_id):
        if id == w1:
            x = True
        elif x and id == w2:
            x = False
            verses.append(word.iloc[i].word_vers_id)
        else:
            x = False
    return verses

In [63]:
def lexn_gloss_lookup(arr):
    return list(lexn[lexn.lexn_gloss.isin(arr)].lexn_id)


In [79]:
def get_interlinear(vers_ids):
    verses = []
    for vers_id in vers_ids:
        vers = gnt[gnt.vers_id==vers_id]        
        vers_ref_abbrev = list(vers.vers_ref_abbrev.unique())[0]
        verses.append([vers_ref_abbrev,]+[(word).replace('¶','') for word,punc in list(zip(vers.word_greek, vers.word_greek_punc))])
        verses.append([vers_ref_abbrev,]+[(word).replace('¶','') for word,punc in list(zip(vers.word_english, vers.word_english_punc))])
    df = pd.DataFrame(verses).fillna('')
    df = df.rename(columns={n:f"word_{n}" for n in df.columns})
    df = df.rename(columns={"word_0":"vers_ref_abbrev"})
    return df

In [72]:
lexn_gloss_lookup(['to talk', 'to speak', 'to say'])

['0012', '0018', '0056', '0252', '2066', '2624']

In [73]:
lexn_gloss_lookup(['God'])

['0013']

In [80]:
get_interlinear(vers_ids)

,vers_ref_abbrev,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,word_11,word_12,word_13,word_14,word_15,word_16,word_17,word_18,word_19,word_20,word_21,word_22,word_23
0,1Co. 8:5,καὶ,γὰρ,εἴπερ,εἰσὶν,λεγόμενοι,θεοὶ,εἴτε,ἐν,οὐρανῷ,εἴτε,ἐπὶ,γῆς,ὥσπερ,εἰσὶν,θεοὶ,πολλοὶ,καὶ,κύριοι,πολλοί,,,,
1,1Co. 8:5,Even,truly,if indeed,there are,[those] called,gods,whether,in,heaven,or,on,earth,as,there are,gods,many,and,Lords,many,,,,
2,2Th. 2:4,ὁ,ἀντικείμενος,καὶ,ὑπεραιρόμενος,ἐπὶ,πάντα,λεγόμενον,θεὸν,ἢ,σέβασμα,ὥστε,αὐτὸν,εἰς,τὸν,ναὸν,τοῦ,θεοῦ,καθίσαι,ἀποδεικνύντα,ἑαυτὸν,ὅτι,ἐστὶν,θεός
3,2Th. 2:4,who,is opposing,and,exalting himself,above,every,so-called,god,or,object of worship,so as for,him,in,the,temple,<the>,of God,to sit down,setting forth,he himself,that,is,God


In [81]:
vers_ids = back_to_back_words('0000', '0114')
vers_ids

['010122',
 '010205',
 '010215',
 '010217',
 '010223',
 '010303',
 '010414',
 '010512',
 '010517',
 '010712',
 '010817',
 '011113',
 '011217',
 '011239',
 '011335',
 '011614',
 '012104',
 '012111',
 '012240',
 '012329',
 '012330',
 '012331',
 '012337',
 '012415',
 '012656',
 '012709',
 '020102',
 '020615',
 '020828',
 '030304',
 '030417',
 '030427',
 '030623',
 '031147',
 '031150',
 '031328',
 '031334',
 '031616',
 '031629',
 '031631',
 '031831',
 '032425',
 '032427',
 '032444',
 '040121',
 '040123',
 '040125',
 '040145',
 '040614',
 '040645',
 '040740',
 '040852',
 '040853',
 '041238',
 '050216',
 '050318',
 '050323',
 '050324',
 '050325',
 '050742',
 '050748',
 '050752',
 '050828',
 '050830',
 '050834',
 '051043',
 '051315',
 '051320',
 '051327',
 '051340',
 '051515',
 '052414',
 '052622',
 '052627',
 '052823',
 '052825',
 '060102',
 '060321',
 '061103',
 '130215',
 '190101',
 '191132',
 '200510',
 '220216',
 '271007',
 '271118',
 '271820',
 '272206',
 '272209']

In [18]:
vers1 = [(word + punc).replace('¶','') for word,punc in list(zip(vers.word_greek, vers.word_greek_punc))]

AttributeError: 'DataFrame' object has no attribute 'word_greek'

In [15]:
verses = [vers1, ]

In [16]:
pd.DataFrame(verses).fillna('')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Δημητρίῳ,μεμαρτύρηται,ὑπὸ,πάντων,καὶ,ὑπὸ,αὐτῆς,τῆς,ἀληθείας·,καὶ,ἡμεῖς,δὲ,"μαρτυροῦμεν,",καὶ,οἶδας,ὅτι,ἡ,μαρτυρία,ἡμῶν,ἀληθής,ἐστιν.
